In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *

In [2]:
#%matplotlib widget
#matplotlib inline
%matplotlib qt
#%matplotlib gtk

import matplotlib.pyplot as plt
import matplotlib.font_manager

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'

from matplotlib.backends.backend_pgf import FigureCanvasPgf
matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
bpC = 150 #CT Compression data Break Point in p-q plane
bpE = 158 #CT Extension data Break Point in p-q plane 

In [4]:
data = load_data(9)
d = convert(data)
P1, P2 = create_P1_and_P2(data,bpC,bpE)
P1_init, P2_init = create_P1_and_P2(data,bpC,bpE)
#print(P1.sol)
#print(P2.sol)
#if P1_init.Vo <= P2_init.Vo :
#    P1 = P2_init
#    P2 = P1_init
#elif P1_init.Vo >= P2_init.Vo :
#    P1 = P1_init
#    P2 = P2_init
#print(P1.sol)

in this situation $\phi_C^1 > \phi_C^2$ in this situation. planes P1 and P2 in zone 0, 2 and 4 do not intersect

if $\phi_E^1 > \phi_E^2$ planes P1 and P2 in zone 1, 3 and 5 do not intersect

In [5]:
print('P1 P2\n{} {}\n{} {}'.format(P1.phyC*180/pi,P2.phyC*180/pi,P1.phyE*180/pi,P2.phyE*180/pi))

P1 P2
19.110608285284137 30.436670603567013
16.69243443141745 30.85512943822601


In [6]:
special_case_phyC = P1.phyC > P2.phyC
special_case_phyE = P1.phyE > P2.phyE

In [7]:
print('P1 < P2 ?\nphyC : {}\nphyE : {}'.format(special_case_phyC,special_case_phyE))

P1 < P2 ?
phyC : False
phyE : False


In [8]:
print('V0 P1 : {}\nV0 P2 : {}'.format(P1.Vo,P2.Vo))

V0 P1 : 181.66296296276897
V0 P2 : 50.01858666022968


In [9]:
offset = None
if special_case_phyC:
    offset = 0
elif special_case_phyE:
    offset = 0
else:
    print("you shouldn't be here")
print('offset = {}'.format(offset))

you shouldn't be here
offset = None


## 3D plot | Calculating intersections

### P1 P2 transition points

In [10]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))

for i in range(6):
    eq_pts_transisiton_P12[(i-offset)%6] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])
    
ones = np.ones((6,3))

pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()
print(pts_transisiton_P12)

TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [ ]:
if offset == 0:
    pts_transisiton_P12 = pts_transisiton_P12[:,1::2]
elif offset == 2:
    pts_transisiton_P12 = pts_transisiton_P12[:,::2]
print(pts_transisiton_P12)
#assert(pts_transisiton_P12.shape==(3,3))

$V_0^2$

In [12]:
eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

### Intersection with far plane

In [13]:
def p_plane_intersection_12(P1,P2,plane_dist,offset):
    p_plane_n = np.ones(3)/np.sqrt(3)
    p_plane_o = p_plane_n*plane_dist
    
    eq = np.zeros((12,3,3))
    b = np.array([[1,1,np.dot(p_plane_n,p_plane_o)]])
    
    for i in range(12):
        eq[i] = np.array([get_plane_normal_12_cycle(P1,P2,i,offset),get_plane_normal_12_cycle(P1,P2,i-1,offset),p_plane_n])
        
    pts = np.linalg.solve(eq,b).transpose()
    return pts
    
    

In [14]:
sig_pts = np.concatenate((P1.pts,P2.pts),axis=1)
all_pts = np.concatenate((sig_pts,pts_transisiton_P12,pts_P2),axis=1) #miss pts_P2
pts_with_largest_coord = 1*all_pts[:,np.argmax(np.max(all_pts, axis=0))]
pts_with_largest_coord *= 1.1

In [15]:
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)
pts_far = p_plane_intersection_12(P1,P2,dist,offset)

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.set_aspect('equal')
ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')

In [17]:
ax.plot(*pts_transisiton_P12[:,:], 'ro',alpha=1)
#ax.plot(*pts_transisiton_P12[:,-1:], 'yo',alpha=1)
ax.plot(*pts_P2,'ko',alpha=1)
ax.plot(*pts_far,'bo',alpha=1)

In [18]:
def get_verts(cord):
    return [list(zip(*cord))]

## Draw P2 planes

In [19]:
for i in range(3):
    cord_l = np.zeros((3,4))
    cord_r = np.zeros((3,4))
    far_pts_inter_P2_idx = i*4
    cord_r[:,0] = pts_P2[:,0]
    cord_r[:,1] = pts_transisiton_P12[:,i] 
    cord_r[:,2] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    cord_l[:,0] = pts_P2[:,0]
    cord_l[:,1] = pts_transisiton_P12[:,(i-1)%3] 
    cord_l[:,2] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 planes

In [20]:
for i in range(3):
    cord_l = np.zeros((3,3))
    cord_r = np.zeros((3,3))
    far_pts_inter_P2_idx = i*4 + 2
    cord_r[:,0] = pts_transisiton_P12[:,i] 
    cord_r[:,1] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,2] = pts_far[:,far_pts_inter_P2_idx]

    cord_l[:,0] = pts_transisiton_P12[:,i] 
    cord_l[:,1] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,2] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [21]:
#ax.set_aspect('equal')
fig.show()

# 2D view

In [26]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [27]:
new_sig_pts = new_base_transform @ sig_pts
new_P2 = new_base_transform @ pts_P2
new_transi = new_base_transform @ pts_transisiton_P12
new_far = new_base_transform @ pts_far

In [28]:
transi_dist = new_transi[2,2]
near_inter = p_plane_intersection_6(P2,transi_dist*0.99)
new_near = new_base_transform @ near_inter

In [29]:
plt.close()
#plt.axis('equal')
plt.plot(new_P2[0,:],new_P2[1,:],'r.')
plt.plot(new_sig_pts[0,:],new_sig_pts[1,:],'k.')
plt.fill(new_far[0,:],new_far[1,:],edgecolor='b',fill=False)
plt.fill(new_near[0,:],new_near[1,:],edgecolor='r',fill=False)

In [26]:
P1.sol

array([41.42024211, 22.75859844, -1.52418406, 22.17600714, 45.41359589,
       38.24425525])

In [27]:
P2.sol

array([41.2051625 , 31.24259624, -0.94555849, 27.18533908, 37.23010391,
       45.29694948])